## Create NWBfile from EEG dataset (single subject)

In [1]:
import os
import sys
import json
import time
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
import pandas as pd
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import BehavioralTimeSeries
from pynwb.ecephys import ElectricalSeries, LFP
from pynwb.misc import Units

from hdmf.backends.hdf5.h5_utils import H5DataIO

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [4]:
mcc = MouseConnectivityCache(resolution=25)
structree = mcc.get_structure_tree()

### Functions

In [5]:
def stim_log_NWBstyle(EEGexp_stim_log, sweep_to_depth_map, sweep_to_epoch_map, invalid_sweeps):
    """
    This function takes an EEGexp-style stim_log and returns a table with more descriptive columns to create an NWB trial table.

    Inputs:
    -------
    EEGexp_stim_log : pandas.DataFrame
        The stim_log associated with a particular EEGexp object.
    sweep_to_depth_map : list of str
        A list containing the stimulation depth associated with each sweep defined in the stim_log.
    sweep_to_epoch_map : list of str
        A list containing the behavioral epoch/state associated with each sweep defined in the stim_log.
    invalid_sweeps : list of int
        A list containing the sweeps excluded from analysis for artifact or off-target stimulation.

    Outputs:
    --------
    trial_table: pandas.DataFrame
        A dataframe with specific columns to create the NWB trial table.
    column_desc_dict: dict
        A dictionary associating the column names to a description of the column information.
    
    """
    stim_type_map = {'biphasic': 'electrical', 'circle': 'visual', 'fullscreen': 'visual'}
    
    ## Apply invalid_sweeps ##
    EEGexp_stim_log['good'] = EEGexp_stim_log.apply(lambda x: False if x.sweep in invalid_sweeps else x.good, axis=1)
    
    ## Initialize the NWBstyle trial table ##
    trial_table = pd.DataFrame({'start_time': EEGexp_stim_log['onset'].values, 'stop_time': EEGexp_stim_log['offset'].values})
    
    ## Adds extra columns describing stimuli ##
    stim_type = []
    stim_desc = []
    stim_curr = []
    stimreg = []
    stimdepth = []
    for _, row in EEGexp_stim_log.iterrows():
        if row.stim_type == 'biphasic':
            stim_type.append(stim_type_map[row.stim_type])
            stim_desc.append(row.stim_type)
            stim_curr.append(row.parameter)
            stimreg.append(sub_meta['stim_region'])
            stimdepth.append(sweep_to_depth_map[row.sweep])
        elif row.stim_type in ['circle', 'fullscreen']:
            stim_type.append(stim_type_map[row.stim_type])
            stim_desc.append(row.parameter + ' ' + row.stim_type)
            stim_curr.append('n/a')
            stimreg.append('n/a')
            stimdepth.append('n/a')
        else:
            stim_type.append('unknown')
            stim_desc.append('n/a')
            stim_curr.append('n/a')
            stimreg.append('n/a')
            stimdepth.append('n/a')
    trial_table['stimulus_type'] = stim_type
    trial_table['stimulus_description'] = stim_desc
    trial_table['estim_current'] = stim_curr
    trial_table['estim_target_region'] = stimreg
    trial_table['estim_target_depth'] = stimdepth
    
    ## Add metadata for each trial ##
    trial_table['behavioral_epoch'] = [sweep_to_epoch_map[x] for x in EEGexp_stim_log['sweep'].values]
    trial_table['is_running'] = ~EEGexp_stim_log['resting_trial'].values
    trial_table['is_valid'] = EEGexp_stim_log['good'].values
    
    ## Column descriptions ##
    column_desc_dict = {
        'stimulus_type': 'type of stimulus delivered',
        'stimulus_description': 'more specific description of stimulus type',
        'estim_current': 'electrical stimulation current (\u03bcA), if applicable',
        'estim_target_region': 'electrical stimulation target region, if applicable',
        'estim_target_depth': 'electrical stimulation target depth (cortical layer), if applicable',
        'behavioral_epoch': 'behavioral epoch when stimulus was delivered',
        'is_running': 'True if mouse is running during trial, running defined as average speed > 0 cm/s between [-0.5, 0.5] s from stimulus onset',
        'is_valid': 'True for valid trials, trials are invalid if they have large electrical artifacts or if electrical stimulation was off target',
    }
    
    return trial_table, column_desc_dict

In [6]:
def get_subexp_metadata(submetaseries, session_start_time):
    subexp_dict = {}
    
    ## Get state label for each sweep ##
    templist = []
    for char in submetaseries['sweep_state'].split(','):
        templist.append(char)
    subexp_dict['sweep_state_map'] = templist
    
    ## Get depth label for each sweep ##
    if ',' in submetaseries['stim_depth']:
        templist = []
        for char in submetaseries['stim_depth'].split(','):
            templist.append(char)
    else:
        templist = [submetaseries['stim_depth']] * len(subexp_dict['sweep_state_map'])
    subexp_dict['sweep_depth_map'] = templist
    
    ## Get EEG bad chs as list ##
    templist = []
    if type(submetaseries['bad_chs']) == str:
        for char in submetaseries['bad_chs'].split(','):
            templist.append(int(char))
    subexp_dict['EEG_bad_chs'] = templist
    
    ## Get invalid sweeps list, if any ##
    templist = []
    if type(submetaseries['invalid_sweeps']) == str:
        for char in submetaseries['invalid_sweeps'].split(','):
            templist.append(int(char))
    subexp_dict['invalid_sweeps'] = templist
    
    ## Get stim target description ##
    if len(np.unique(subexp_dict['sweep_depth_map'])) > 1:
        layer_tag = '(superficial and deep layers)'
    elif len(np.unique(subexp_dict['sweep_depth_map'])) == 1:
        layer_tag = '({} layers)'.format(np.unique(subexp_dict['sweep_depth_map'])[0])
    else:
        layer_tag = '(unknown depth)'
    subexp_dict['stim_notes'] = "single pulse electrical stimuli targeted to {} {}".format(submetaseries['stim_region'], layer_tag)
    
    sub_DOB = submetaseries['DOB'].to_pydatetime().replace(tzinfo=tzlocal())
    subexp_dict['subject_age'] = "P{:d}D".format((session_start_time - sub_DOB).days)
    
    return subexp_dict

### Load mouse/experiment metadata .xlsx file

In [7]:
startclock = time.time()

In [8]:
subject_metadata = pd.read_excel(
    r"E:\NWB_testing\all_subject_metadata.xlsx", sheet_name='experiment_metadata', dtype={'Mouse': str, 'valid_session': bool}
)

### Load EEG electrodes locations

In [9]:
EEGch_locs = pd.read_csv(r"E:\NWB_testing\EEG_electrodes_info.csv")
# EEGch_locs.head()

## Set metadata

In [10]:
institution = "Allen Institute" # MindScope Program?
publications = "https://doi.org/10.7554/eLife.84630.1"
experimenters = [
    "Claar, Leslie D",
    "Rembado, Irene",
    "Kuyat, Jacqulyn R",
    "Russo, Simone",
    "Marks, Lydia C",
    "Olsen, Shawn R",
    "Koch, Christof"
] # experimenter: Claar, Leslie D - can we include multiple authors? Can be a str or list
exp_desc = "in vivo electrophysiology in a head-fixed mouse during cortical electrical microstimulation"
keywords = ['EEG', 'cortico-thalamo-cortical', 'electrophysiology', 'anesthesia']

## Load experiment

In [11]:
recfolder = r'F:\EEG_exp\mouse569073\estim_vis_2021-04-15_10-27-22\experiment1\recording1'
exp = EEGexp(recfolder, preprocess=False, make_stim_csv=False)

Experiment type: electrical and sensory stimulation


In [12]:
if np.any([True for xd in exp.experiment_data if 'probe' in xd]):
    session_desc = "EEG and Neuropixels recording during wakefulness and isoflurane anesthesia"
else:
    session_desc = "EEG recording during wakefulness and isoflurane anesthesia"
print(session_desc)

EEG and Neuropixels recording during wakefulness and isoflurane anesthesia


**Set NWB file output location**

In [13]:
NWB_output_dir = os.path.dirname(os.path.dirname(recfolder))

**Get session_start_time from sync file**

In [14]:
sync_data = exp._load_sync_dataset()
session_start_time = datetime.strptime(sync_data.meta_data['start_time'], '%Y-%m-%d %H:%M:%S.%f').replace(tzinfo=tzlocal())
print(session_start_time)

2021-04-15 10:26:36.611000-07:00


**Get this subject's metadata from the excel file**

In [15]:
sub_meta = subject_metadata[
    (subject_metadata['Mouse'] == exp.mouse) &
    (subject_metadata['exp_folder'] == os.path.basename(os.path.dirname(exp.experiment_folder)))
].squeeze() # .squeeze() makes this into a series

In [16]:
sub_data_dict = get_subexp_metadata(sub_meta, session_start_time)

In [17]:
sub_data_dict.keys()

dict_keys(['sweep_state_map', 'sweep_depth_map', 'EEG_bad_chs', 'invalid_sweeps', 'stim_notes', 'subject_age'])

In [18]:
print(sub_data_dict['sweep_state_map'])
print(sub_data_dict['sweep_depth_map'])
print('bad chs: ', sub_data_dict['EEG_bad_chs'])
print('bad sweeps: ', sub_data_dict['invalid_sweeps'])
print(sub_data_dict['stim_notes'])
print(sub_data_dict['subject_age'])

['awake', 'isoflurane', 'recovery', 'recovery']
['deep', 'deep', 'deep', 'deep']
bad chs:  [3, 4, 13]
bad sweeps:  []
single pulse electrical stimuli targeted to MOs (deep layers)
P128D


**Create session id (mouse-expdate) and identifier**
<br>"identifier" should be unique to this NWBfile (and does not need to be easily human-readable)

In [19]:
session_id = "{}-{}".format(exp.mouse, session_start_time.strftime("%Y%m%d"))
print(session_id)

569073-20210415


In [20]:
file_id = session_id + "-test03_comp"

## Initialize the NWB file
Other optional fields exist, see: https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.

In [21]:
nwbfile = NWBFile(
    session_description = session_desc,
    identifier = file_id,
    session_start_time = session_start_time,
    experimenter = experimenters,
    experiment_description = exp_desc, # optional
    session_id = session_id,
    institution = institution,
    keywords = keywords,
    related_publications = publications,
    stimulus_notes = sub_data_dict['stim_notes'],
)
print(nwbfile)

root pynwb.file.NWBFile at 0x2127567545800
Fields:
  experiment_description: in vivo electrophysiology in a head-fixed mouse during cortical electrical microstimulation
  experimenter: ['Claar, Leslie D' 'Rembado, Irene' 'Kuyat, Jacqulyn R' 'Russo, Simone'
 'Marks, Lydia C' 'Olsen, Shawn R' 'Koch, Christof']
  file_create_date: [datetime.datetime(2023, 3, 12, 15, 31, 40, 465416, tzinfo=tzlocal())]
  identifier: 521885-20200709-test03_comp
  institution: Allen Institute
  keywords: ['EEG' 'cortico-thalamo-cortical' 'electrophysiology' 'anesthesia']
  related_publications: ['https://doi.org/10.7554/eLife.84630.1']
  session_description: EEG recording during wakefulness and isoflurane anesthesia
  session_id: 521885-20200709
  session_start_time: 2020-07-09 14:17:02.725000-07:00
  stimulus_notes: single pulse electrical stimuli targeted to MOs (superficial layers)
  timestamps_reference_time: 2020-07-09 14:17:02.725000-07:00



## Add subject info
Other optional fields exist, see: https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.Subject.

In [22]:
nwbfile.subject = Subject(
    age = sub_data_dict['subject_age'],
    description = "mouse" + exp.mouse,
    sex = sub_meta['Sex'],
    species = "Mus musculus",
    subject_id = exp.mouse,
    strain = sub_meta['Strain'],
)

## Add running speed

In [23]:
speed, speed_times = exp.load_running()
# print(speed.shape)
# print(speed_times[0])
# print(speed_times[-1])

speed_with_samprate = TimeSeries(
    name = "running_speed",
    data = speed,
    unit = "cm/s",
    starting_time = speed_times[0], # must be a float, not int
    rate = 100.0, # sampling rate must be a float
    description = "running speed data, computed from wheel angular velocity"
)

In [24]:
behavioral_time_series = BehavioralTimeSeries(
    time_series = speed_with_samprate, name = "BehavioralTimeSeries", # can rename, but should not
)
behavior_module = nwbfile.create_processing_module(name="behavior", description="processed behavioral data")
behavior_module.add(behavioral_time_series)

BehavioralTimeSeries pynwb.behavior.BehavioralTimeSeries at 0x2057368625864
Fields:
  time_series: {
    running_speed <class 'pynwb.base.TimeSeries'>
  }

## Add EEG

In [45]:
if 'EEG' in session_desc:
    print('This recording has EEG.')

This recording has EEG.


Add extra columns to electrodes table

In [25]:
nwbfile.add_electrode_column(name = "is_data_valid", description = "True for electrodes/channels with usable data")

In [26]:
# Specifically for NPXs, must add here though #
nwbfile.add_electrode_column(name = "probe_vertical_position", description = "length-wise position of electrode/channel on device (microns)")
nwbfile.add_electrode_column(name = "probe_horizontal_position", description = "width-wise position of electrode/channel on device (microns)")

**Add EEG device**

In [27]:
eegdevice = nwbfile.create_device(name = "EEG array", description = "H32 Mouse EEG (30-ch)", manufacturer = "Neuronexus")

**Add 30 electrodes in one group**

In [28]:
EEG_elec_group = nwbfile.create_electrode_group(
    name = "EEG array", description = "30-ch surface grid", device = eegdevice, location = "skull surface, both hemispheres",
)

In [30]:
ch_valid = [True if x not in sub_data_dict['EEG_bad_chs'] else False for x in range(len(EEGch_locs))]
for chi, row in EEGch_locs.iterrows():
    nwbfile.add_electrode(
        x = float(row.x),
        y = float(row.y),
        z = float(row.z),
        location = row.location,
        group = EEG_elec_group,
        reference = sub_meta['Reference'],
        is_data_valid = ch_valid[chi],
        probe_vertical_position = -1, # "n/a" could be causing a problem with other float values
        probe_horizontal_position = -1,
    )

*All electrodes will be added to the same DynamicTable, EEG and all NPXs.* Will need to keep track, so the DynamicTableRegion can be assigned accordingly.

**Create a DynamicTableRegion for the EEG electrodes**

In [28]:
EEGchs = list(np.arange(len(EEGch_locs)))
EEG_table_region = nwbfile.create_electrode_table_region(region = EEGchs, description = "EEG electrodes")

**Add EEG data with timestamps**

In [29]:
## Try from continuous file ##
eeg_data = exp._memmap_EEGdata()
eeg_times = np.load(exp.ephys_params['EEG']['timestamps'])

In [30]:
## With H5DataIO compression ##
EEG_electrical_series = ElectricalSeries(
    name = "ElectricalSeriesEEG", # this is the key used to call up the data within the NWBfile, BP: ElectricalSeriesFromEEG, etc.
    data = H5DataIO(data=eeg_data, compression=True),
    electrodes = EEG_table_region,
    timestamps = H5DataIO(data=eeg_times, compression=True),
#     resolution = , # float, smallest meaningful difference between values in data (in V)
    conversion = exp.ephys_params['EEG']['bit_volts'] * 1E-6, # float, scalar to multiply data by to convert it to Volts
    description = "voltage measured over time and associated timestamps from EEG array",
)

In [31]:
nwbfile.add_acquisition(EEG_electrical_series)

## Add NPX to electrodes table

In [32]:
probe_list = [x.replace('_sorted', '') for x in exp.experiment_data if 'probe' in x]
if len(probe_list) == 0:
    print(' This experiment has no probe data, not adding NPX devices to NWBfile.\n')
#     continue
print(probe_list)

['probeB', 'probeC', 'probeF']


In [33]:
probe_locs = np.ones((len(probe_list)), dtype=bool)
for pbi, probei in enumerate(probe_list):
    with open(exp.ephys_params[probei]['probe_info']) as data_file:
        data = json.load(data_file)
    if 'area_ch' not in data.keys():
        probe_locs[pbi] = False
if ~probe_locs.any():
    print(' This experiment has no probe locations, not adding NPX devices to NWBfile.\n')

In [34]:
unit_col_desc_map = {
    'location': 'the location of unit within the brain (CCF acronym)',
    'x': 'the x coordinate of the position (CCF, +x is posterior)',
    'y': 'the y coordinate of the position (CCF, +y is inferior)',
    'z': 'the z coordinate of the position (CCF, +z is right)',
    'isi_violations': 'an estimate of the relative firing rate of hypothetical neurons generating inter-spike-interval violations',
    'amplitude_cutoff': 'an estimate of the fraction of spikes below the spike detection threshold',
    'presence_ratio': 'the fraction of time the unit was present during the experiment (ranges from 0 to 0.99)',
    'waveform_duration': 'the mean duration (s) of detected spiking events',
}

for coln, cold in unit_col_desc_map.items():
    nwbfile.add_unit_column(name = coln, description = cold)

In [35]:
try:
    electrode_counter = len(nwbfile.electrodes)
except TypeError:
    electrode_counter = 0

probe_NWB_objects = {}

for pbi, probei in enumerate(probe_list):
    if probe_locs[pbi]:
        probe_NWB_objects[probei] = {}
        if '-PXI-' in exp.ephys_params[probei]['lfp_continuous']:
            dev_desc = "Neuropixels 1.0 probe"
            ref_type = "tip reference configuration"
        elif '-3a-' in exp.ephys_params[probei]['lfp_continuous']:
            dev_desc = "Neuropixels 3a probe"
            ref_type = "external reference configuration"
        else:
            dev_desc = "Neuropixels probe"
            ref_type = "external reference configuration"

        ## Create device ##
        probe_NWB_objects[probei]['device'] = nwbfile.create_device(
            name = probei, description = dev_desc, manufacturer = "imec",
        )

        ## Create electrode group ##
        probe_NWB_objects[probei]['elec_group'] = nwbfile.create_electrode_group(
            name = probei,
            description = "Neuropixels probe " + probei[-1],
            device = probe_NWB_objects[probei]['device'],
            location = "see electrode locations",
        )

        ## Load probe_info.json ##
        with open(exp.ephys_params[probei]['probe_info']) as data_file:
            data = json.load(data_file)
        npx_allch = np.array(data['channel'])
        surface_ch = int(data['surface_channel'])
        ref_mask = np.array(data['mask'])
        npxch_valid = [True if ref_mask[x] and x <= surface_ch else False for x in npx_allch]
        vert_pos = np.array(data['vertical_pos'])
        horiz_pos = np.array(data['horizontal_pos'])
        ch_locs = np.array(data['area_ch'])
        ## Convert CCF coords to 25 um resolution, if needed ##
        if sub_meta['Histology'] == 'slices':
            ccf_coords = np.round(np.array(data['ccf_coord_ch']) * (10 / 25))
        elif sub_meta['Histology'] == 'TC':
            ccf_coords = np.array(data['ccf_coord_ch']).astype(float)
        else:
            print('No info about histology type.')

        for npxchi in npx_allch:
            ## Strip the layer tag from cortical areas and assign "none" to chs not in brain ##
            try:
                if 315 in structree.get_structures_by_acronym([ch_locs[npxchi]])[0]['structure_id_path']:
                    chi_loc = structree.parents([structree.get_structures_by_acronym([ch_locs[npxchi]])[0]['id']])[0]['acronym']
                else:
                    chi_loc = ch_locs[npxchi]
            except KeyError:
                chi_loc = 'none'

            nwbfile.add_electrode(
                x = ccf_coords[npxchi, 0],
                y = ccf_coords[npxchi, 1],
                z = ccf_coords[npxchi, 2],
                location = chi_loc,
                group = probe_NWB_objects[probei]['elec_group'],
                reference = ref_type,
                is_data_valid = npxch_valid[npxchi],
                probe_vertical_position = vert_pos[npxchi],
                probe_horizontal_position = horiz_pos[npxchi],
            )
            
        ## Create probe DynamicTableRegion ##
        probechs = list(np.arange(electrode_counter, electrode_counter + len(npx_allch)))
        probe_NWB_objects[probei]['elec_table_region'] = nwbfile.create_electrode_table_region(
            region = probechs, description = "{} electrodes".format(probei),
        )
        electrode_counter += len(npx_allch)
        
        ## Add the LFP data ##
        lfp_ts = np.load(exp.ephys_params[probei]['lfp_timestamps'])
        lfp_mm = np.memmap(
            exp.ephys_params[probei]['lfp_continuous'], dtype='int16',
            shape=(lfp_ts.size, exp.ephys_params[probei]['num_chs']), mode='r'
        )
        
        ## Method with H5DataIO compression ##
        probe_NWB_objects[probei]['ElecSeries'] = ElectricalSeries(
            name = "ElectricalSeries" + probei, # BP: ElectricalSeriesFromEEG, etc.
            data = H5DataIO(data=lfp_mm, compression=True),
            electrodes = probe_NWB_objects[probei]['elec_table_region'],
            timestamps = H5DataIO(data=lfp_ts, compression=True),
            conversion = exp.ephys_params[probei]['bit_volts'] * 1E-6,
            description = "voltage measured over time and associated timestamps from Neuropixels" + probei,
        )
        probe_NWB_objects[probei]['LFPSeries'] = LFP(
            electrical_series = probe_NWB_objects[probei]['ElecSeries'], name = "LFP" + probei
        )
        nwbfile.add_acquisition(probe_NWB_objects[probei]['LFPSeries'])
        
        ## Add probe units ##
        select_units, peak_chs, unit_metrics = exp.get_probe_units(probei)

        all_spike_times = np.load(exp.ephys_params[probei]['spike_times'])
        all_spike_clusters = np.load(exp.ephys_params[probei]['spike_clusters'])
        all_spike_waveforms = np.load(exp.ephys_params[probei]['waveforms']) # [clusters, chs, time] (uV)

        for ii, (indi, row) in enumerate(unit_metrics.iterrows()):
            uniti = row.cluster_id
            spikesi = np.squeeze(all_spike_times[all_spike_clusters == uniti])

            ## Strip the layer tag from cortical areas and assign "none" to chs not in brain ##
            try:
                if 315 in structree.get_structures_by_acronym([row.area])[0]['structure_id_path']:
                    uniti_loc = structree.parents([structree.get_structures_by_acronym([row.area])[0]['id']])[0]['acronym']
                else:
                    uniti_loc = row.area
            except KeyError:
                uniti_loc = 'none'

            ## Convert CCF coords to 25 um resolution, if needed ##
            unit_coords = [int(x) for x in row.ccf_coord.replace('[','').replace(']','').replace(' ','').split(',')]
            if sub_meta['Histology'] == 'slices':
                unit_coords = np.round(np.array(unit_coords) * (10 / 25))
            elif sub_meta['Histology'] == 'TC':
                unit_coords = np.array(unit_coords).astype(float)

            ## Add unit to NWBfile ##
            nwbfile.add_unit(
                spike_times = spikesi,
                electrodes = [probe_NWB_objects[probei]['elec_table_region'][row.peak_channel].index[0]],
                electrode_group = probe_NWB_objects[probei]['elec_group'],
                waveform_mean = all_spike_waveforms[uniti, row.peak_channel, :] * 1E-6, # now in V
                location = uniti_loc,
                x = unit_coords[0],
                y = unit_coords[1],
                z = unit_coords[2],
                isi_violations = row.isi_viol,
                amplitude_cutoff = row.amplitude_cutoff,
                presence_ratio = row.presence_ratio,
                waveform_duration = row.duration,
            )

    else:
        print(' This probe has no locations, not adding it to NWBfile.\n')


## Add stimulus log info

In [21]:
stim_log = pd.read_csv(exp.stimulus_log_file).astype({'parameter': str})

In [30]:
480+360+120

960

In [32]:
stim_log[475:485]

,stim_type,parameter,onset,offset,duration,sweep,good,mean_speed,resting_trial
475,circle,white,2489.83403,2490.08341,0.24938,0,True,0.0,True
476,circle,white,2494.30442,2494.55383,0.24941,0,True,0.0,True
477,circle,white,2498.47462,2498.72397,0.24935,0,True,0.0,True
478,circle,white,2502.92832,2503.17771,0.24939,0,True,0.0,True
479,circle,white,2507.64894,2507.89832,0.24938,0,True,0.0,True
480,biphasic,40,3769.25855,3769.25895,0.00040,1,True,0.0,True
481,biphasic,20,3772.79157,3772.79197,0.00040,1,True,0.0,True
482,biphasic,40,3776.85320,3776.85360,0.00040,1,True,0.0,True
483,biphasic,70,3781.23548,3781.23588,0.00040,1,True,0.0,True
484,biphasic,70,3785.40509,3785.40549,0.00040,1,True,0.0,True


In [24]:
np.unique(stim_log['sweep'].values)

array([0, 1, 2, 3, 4], dtype=int64)

In [25]:
sub_data_dict['sweep_state_map']

['awake', 'isoflurane', 'recovery', 'recovery']

**Take stim_log and add NWBstyle info to it**

In [23]:
trial_table, col_desc_map = stim_log_NWBstyle(
    stim_log, sub_data_dict['sweep_depth_map'], sub_data_dict['sweep_state_map'], sub_data_dict['invalid_sweeps']
)

IndexError: list index out of range

**Add trials to NWBfile**

In [31]:
## Add new columns to trial table ##
for coln, cold in col_desc_map.items():
    nwbfile.add_trial_column(name = coln, description = cold)
    
## Now add each trial ##
for _, row in trial_table.iterrows():
    row_dict = row.to_dict()
    nwbfile.add_trial(**row_dict)

## Add behavioral epochs/states

Some mice do not have the iso signal...

In [27]:
try:
    iso_induction, iso_maintenance = exp.load_iso_times()
except IndexError:
    print('No iso signal :(')
    iso_induction = (
        trial_table[trial_table['behavioral_epoch'] == 'awake']['stop_time'].values[-1],
        trial_table[trial_table['behavioral_epoch'] == 'isoflurane']['start_time'].values[0]
    )
    iso_maintenance = (
        trial_table[trial_table['behavioral_epoch'] == 'isoflurane']['start_time'].values[0],
        trial_table[trial_table['behavioral_epoch'] == 'isoflurane']['stop_time'].values[-1]
    )

In [29]:
iso_induction

(2630.051094753146, 5919.701094753145)

In [28]:
iso_maintenance

(5919.7110947531455, 6719.221094753146)

In [40]:
nwbfile.add_epoch(
    start_time = iso_induction[0],
    stop_time = iso_induction[-1],
    tags = ['isoflurane_induction']
)
nwbfile.add_epoch(
    start_time = iso_maintenance[0],
    stop_time = iso_maintenance[-1],
    tags = ['isoflurane_anesthesia']
)

In [41]:
midclock = time.time()
print('Time to package: {:.2f} s'.format(midclock - startclock))

Time to package: 415.23 s


## Write the NWBfile

Once you have finished adding all of your data to the :py:class:`~pynwb.file.NWBFile`,
write the file with :py:class:`~pynwb.NWBHDF5IO`.

In [42]:
my_test_file = os.path.join(NWB_output_dir, file_id + r'.nwb')

In [43]:
with NWBHDF5IO(my_test_file, "w") as io:
    io.write(nwbfile, cache_spec=True)
stopclock = time.time()
print('Time to write NWB file: {:.2f} min'.format((stopclock - midclock) / 60))

Time to write NWB file: 34.50 min


# Practice

#### TEST: Add units with spiketimes

**Below is development for the stim_log conversion function**

## Inspect the NWBfile